# Intro to object oriented Python

Everything in Python is an object: variables, raw types, functions, everything.

Sometimes it's useful to organize code around data — to think of a type of data as owning its own functions and metadata. In essence, we store data along with relevant functions (methods) in one 'thing' — an object. You can also think of this as writing your own 'types'.

You can also build very complicated code this way, so be careful! Don't dive into writing classes until you've used Python for a while and start to see how most people use objects in Python. 

This notebook builds on the workflow in [Intro to PYthon](Intro_to_Python.ipynb), so you shoudl be sure to look at that notebook first.

## Types

Python has several types, many of which you're already familiar with.

In [ ]:
type(5)

In [ ]:
type('fifteen'), type('15')

'fifteen' is an **instance** of the class **str**.

Most of these objects have methods, which are just functions that 'belong' to the class:

In [ ]:
'fifteen'.find('t')  # Call the method `find` on the string

In [ ]:
lst = [1,3,4,5,7,8]
lst.append(1000)
lst

## A simple class

In [ ]:
layers = [0.23, 0.34, 0.45, 0.25, 0.23, 0.35]

Start with only one special method: `__init__()` which is required anyway.

Inside `__init__()` we'll only define one **attribute**, `self.layers`.

In [ ]:
import numpy as np

class Layers(object):  # The 'new' Python base class — gives us some useful basic features.
    
    def __init__(self, layers):  # All methods take `self`, which is 'this' instance

        # Just make sure we end up with an array
        self.layers = np.array(layers)
        
        # __init__ can not return anything, and most people don't include a `return` line.

Now we can **instantiate** an instance of the class **Layers**

In [ ]:
l = Layers(layers=layers)

In [ ]:
l

In [ ]:
l.layers

At this point, we might as well just have an array, like we had before. So let's add another attribute, let's give the object a 'name':

In [ ]:
class Layers(object):
    
    def __init__(self, layers, label=None):  # We use a default value of None

        self.layers = np.array(layers)
        self.label = label or "My log"  # This trick substitutes anything evaluating as False with 'My log'

In [ ]:
l = Layers(layers, label='Well 1')

In [ ]:
l.label

## Magic methods

Let's find out how big our fancy `Layers` object is...

In [ ]:
len(l)

Dammit!

The class inherited from `object` when we defined it, and it doesn't know how to apply `len` to our thing. We could do this...

In [ ]:
len(l.layers)

But sometimes we'd like something to 'just work' — maybe later we're going to pass our instances to something that can take lists or our new objects, and that thing might call `len()` on our object. 

That's why this also doesn't help:

In [ ]:
class Layers(object):
    def __init__(self, layers, label=None):
        self.layers = np.array(layers)
        self.label = label or "My log"
        self.length = self.layers.size

What's more, storing length in an attribute like this is unexpected...

So we tell it with a so-called 'dunder' (double-underscore) or ['magic' method](https://docs.python.org/3/reference/datamodel.html) (not to be confused with IPython magics). That way, we can give our object the same interface as typical Python objects.

In [ ]:
class Layers(object):
    def __init__(self, layers, label=None): 
        self.layers = np.array(layers)
        self.label = label or "My log"

    def __len__(self):
        return len(self.layers)

In [ ]:
l = Layers(layers, label='Well 1')
len(l)

Another useful magic method is `__repr__()`. Right now all we see when we do `l` on its own is:

In [ ]:
l

A similar method `__str__()` defines what happens if we cast the object to a `str` type — which is also what happens to it if we call `print()` on it.

<div class="alert alert-success">
<h3>Exercise:</h3>

- Try writing a `__repr__()` method.
- It should return `Layers(n=6)` when you type the name of a 6-layer instance on the command line.
</div>

In [ ]:
class Layers(object):
    def __init__(self, layers, label=None): 
        self.layers = np.array(layers)
        self.label = label or "My log"

    def __len__(self):
        return len(self.layers)
    
    def __repr__(self):
        
        # Your code here.

A handy extra magic method in Jupyter notebook is `_repr_html_()` which tells the notebook what to use for `repr()` — it overrides `__repr__()` when you're in the notebook.

In [ ]:
class Layers(object):
    def __init__(self, layers, label=None):
        self.layers = np.array(layers)
        self.label = label or "My log"
        self.length = self.layers.size
        
    def __len__(self):
        return len(self.layers)

    def _repr_html_(self):
        rows = "<tr><th>{}</th></tr>".format(self.label)
        rows += "<tr><td>"
        layer_strs = [str(i) for i in self.layers]
        rows += "</td></tr><tr><td>".join(layer_strs)
        rows += "</td></tr>"
        html = "<table>{}</table>".format(rows)
        return html

In [ ]:
l = Layers(layers, label='Well 1')
l

<div class="alert alert-success">
<h3>Exercise</h3>

- Try writing a method that allows you to do layers1 + layers2. You will need to define an `__add__()` method. It should return the result ofthe concatenation.
- Use `np.hstack([arr1, arr2])` to stack the arrays containing the data. Form some kind of new label.
</div>

In [ ]:
class Layers(object):
    def __init__(self, layers, label=None):
        self.layers = np.array(layers)
        self.label = label or "My log"
        self.length = self.layers.size
        
    def __len__(self):
        return len(self.layers)

    
    def _repr_html_(self):
        rows = "<tr><th>{}</th></tr>".format(self.label)
        rows += "<tr><td>"
        layer_strs = [str(i) for i in self.layers]
        rows += "</td></tr><tr><td>".join(layer_strs)
        rows += "</td></tr>"
        html = "<table>{}</table>".format(rows)
        return html

## Constructor methods

It can be handy to have several ways to instantiate a class, rather than knowing you'll pass a list (say). 

Let's make a CSV file and write a special method to read those. Even though this is only a 1D dataset, I'll make it more interesting — and more like what you'll normally encounter — by assuming we're reading lots of rows from the CSV.

In [ ]:
import csv

with open('../data/my_layers.csv', 'w') as f:
    writer = csv.writer(f)
    for row in layers:
        writer.writerow([row])  # Have to form a list b/c it's just a 1D array.

In [ ]:
class Layers(object):
    def __init__(self, layers, label=None):
        self.layers = np.array(layers)
        self.label = label or "My log"
        self.length = self.layers.size
        
    def __len__(self):
        return len(self.layers)

    
    def _repr_html_(self):
        rows = "<tr><th>{}</th></tr>".format(self.label)
        rows += "<tr><td>"
        layer_strs = [str(i) for i in self.layers]
        rows += "</td></tr><tr><td>".join(layer_strs)
        rows += "</td></tr>"
        html = "<table>{}</table>".format(rows)
        return html
    
    # This decorator indicates to Python that this method handles classes not instances.
    @classmethod
    def from_csv(cls, filename, column=0):  # Takes cls not self.
        layer_list = []
        with open(filename, 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                layer_list.append(row[column])
        return cls(layer_list)  # Returns an instance of the class.

In [ ]:
l = Layers.from_csv('../data/my_layers.csv', column=0)
l

<div class="alert alert-success">
<h3>Exercises</h3>

- Can you write a method to compute reflection coefficients for these layers?
- Can you write a method to plot the `Layers` instance?
</div>

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

class Layers(object):
    def __init__(self, layers, label=None):
        self.layers = np.array(layers)
        self.label = label or "My log"
        self.length = self.layers.size
        
    def __len__(self):
        return len(self.layers)

    #!--
    def __repr__(self):
        return 'Layers(length={})'.format(len(self))
    
    def __add__(self, other):
        new_layers = np.hstack([self.layers, other.layers])
        new_label = "{} + {}".format(self.label, other.label)
        return Layers(new_layers, label=new_label)
    #--!
    
    def _repr_html_(self):
        rows = "<tr><th>{}</th></tr>".format(self.label)
        rows += "<tr><td>"
        layer_strs = [str(i) for i in self.layers]
        rows += "</td></tr><tr><td>".join(layer_strs)
        rows += "</td></tr>"
        html = "<table>{}</table>".format(rows)
        return html
    
    # This decorator indicates to Python that this method handles classes not instances.
    @classmethod
    def from_csv(cls, filename, column=0):  # Takes cls not self.
        layer_list = []
        with open(filename, 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                layer_list.append(row[column])
        return cls(layer_list)  # Returns an instance of the class.   
    


In [ ]:
l = Layers(layers, label='Well 1')
l.rcs()

In [ ]:
l.plot()

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2016</p>
</div>